<a href="https://colab.research.google.com/github/MorrisSimons/SCH_PA1454_Pytorch_CPU_GPU_preformance_test/blob/main/Pytorch_CPU_GPU_preformance_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time
import psutil
import os

# Function to measure CPU RAM usage
def get_cpu_ram_usage():
    process = psutil.Process(os.getpid())
    ram_usage = process.memory_info().rss / (1024 ** 2)  # Convert to MB
    return ram_usage

# Function to measure GPU RAM usage
def get_gpu_ram_usage():
    if torch.cuda.is_available():
        return torch.cuda.memory_allocated() / (1024 ** 2)  # Convert to MB
    else:
        return 0

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Hyperparameters
num_epochs = 1
batch_size = 100
learning_rate = 0.001

# MNIST dataset
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = torchvision.datasets.MNIST(root='./data',
                                           train=True,
                                           transform=transform,
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data',
                                          train=False,
                                          transform=transform,
                                          download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

# Simple CNN model
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2),
            nn.ReLU())
        self.pool = nn.MaxPool2d(2)
        self.fc = nn.Linear(16 * 14 * 14, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.pool(out)
        out = out.view(-1, 16 * 14 * 14)
        out = self.fc(out)
        return out

model = ConvNet().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Measure initial RAM usage
initial_cpu_ram = get_cpu_ram_usage()
initial_gpu_ram = get_gpu_ram_usage()

# Start time
start_time = time.time()

# Training loop
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):

        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Ensure all GPU operations are complete
if device.type == 'cuda':
    torch.cuda.synchronize()

end_time = time.time()
training_time = end_time - start_time

# Measure final RAM usage
final_cpu_ram = get_cpu_ram_usage()
final_gpu_ram = get_gpu_ram_usage()

# Save the model
model_path = 'model.pth'
torch.save(model.state_dict(), model_path)

# Get disk space used by the model
model_size = os.path.getsize(model_path) / (1024 ** 2)  # Convert to MB

# Function to evaluate the model
def evaluate_model(model, test_loader):
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:

            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Calculate accuracy
test_accuracy = evaluate_model(model, test_loader)

# Print performance metrics
print(f"\nPerformance Metrics:")
print(f"Training Time: {training_time:.2f} seconds")
print(f"CPU RAM Usage: {final_cpu_ram - initial_cpu_ram:.2f} MB")
if device.type == 'cuda':
    print(f"GPU RAM Usage: {final_gpu_ram - initial_gpu_ram:.2f} MB")
else:
    print("GPU not available.")
print(f"Model Disk Size: {model_size:.2f} MB")
print(f"Test Accuracy: {test_accuracy:.2f}%")


Using device: cpu

Performance Metrics:
Training Time: 32.88 seconds
CPU RAM Usage: -39.96 MB
GPU not available.
Model Disk Size: 0.12 MB
Test Accuracy: 96.69%
